In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Dense
from keras import Input
from keras.layers import Concatenate
from keras.utils import plot_model

In [2]:
# The data describe the characteristics of a house (surface, number of rooms...)
# and you should predict its price

# The metrics for the ranking will be based on the mean square error ('mse')

df_train = pd.read_csv("Train.csv")
df_train.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,77747,50,1,1,72,98938,8,10,2014,0,1,4829,4204,455,0,3,7786197.4
1,28459,52,0,0,4,50114,1,7,2012,1,1,3570,9363,318,0,6,2846923.2
2,34668,69,1,1,12,17366,8,7,2002,0,1,3476,7549,503,1,7,3475230.2
3,64335,4,0,0,83,65939,6,9,2020,1,0,7301,6989,802,0,1,6435779.0
4,22875,86,1,0,95,8661,5,7,1994,0,0,6394,2119,542,0,7,2295511.4


In [3]:
df_test = pd.read_csv("Test.csv")
df_test.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,92649,31,0,0,77,56098,10,8,1991,1,1,6811,2656,429,1,8,9271774.1
1,42485,1,0,0,40,23782,7,7,2017,0,1,974,9553,426,0,3,4251851.7
2,65867,37,0,0,92,67725,6,2,1994,1,1,3054,4650,998,1,10,6593211.4
3,2372,41,1,0,5,60320,1,1,2003,0,0,8041,7294,736,1,4,241014.3
4,60514,46,1,1,35,76413,2,1,2019,1,0,9227,737,340,0,8,6059360.9


In [94]:
# Separate input data/output data and store into numpy array

X_train = np.array(df_train.drop('price', axis=1))
y_train = np.array(df_train['price'])/10000

X_test = np.array(df_test.drop('price', axis=1))
y_test = np.array(df_test['price'])/10000

In [95]:
min, max = np.min(X_train, axis = 0), np.max(X_train, axis = 0)

X_train = (X_train - min)/(max - min)
X_test = (X_test - min)/(max - min)

In [5]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(8000, 16) (8000,)
(1000, 16) (1000,)


In [75]:
# define model

# Input layer
x0 = [Input(shape = (1, )) for _ in range (16)]

# Hiddens layer
x1 = [Dense(3, input_dim = 1, activation = "relu")(x) for x in x0]

x2 = [Dense(3, activation = "relu")(x) for x in x1]

x3 = [Dense(3, activation = "relu")(x) for x in x2]

x3_temp = x3.copy()

x4 = [Concatenate()([x3[i], x3[i + 1]]) for i in range (0, len(x3), 2)]

x5 = [Dense(6, activation = "relu")(x) for x in x4]

x6 = [Dense(5, activation = "relu")(x) for x in x5]

x7 = [Concatenate()([x6[i], x6[i + 1]] + x3_temp[2*i : int(4*(i/2 + 1))]) for i in range (0, len(x6), 2)]

x8 = [Dense(8, activation = "relu")(x) for x in x7]

x9 = [Dense(8, activation = "relu")(x) for x in x8]

x10 = [Concatenate()([x9[i], x9[i + 1]]) for i in range (0, len(x9), 2)]

x11 = [Dense(16, activation = "relu")(x) for x in x10]

x12 = Concatenate()(x11)

x13 = Dense(256, activation = "relu")(x12)

# # Output layer
x14 = Dense(1)(x13)

# Create model
model = Model(inputs = x0, outputs = x14)

plot_model(model, show_shapes = True, to_file = "model_graph_test.png") ;

model.compile(optimizer = 'rmsprop', loss = ["mse"], metrics = ["mse"])

model.fit([X_train[:, i] for i in range (16)], y_train, epochs = 3, batch_size = 10, validation_data = ([X_test[:, i] for i in range (16)], y_test))

Epoch 1/3
800/800 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 9582.5137 - mse: 9582.5137 - val_loss: 1256.0455 - val_mse: 1256.0455
Epoch 2/3
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 584.5867 - mse: 584.5867 - val_loss: 560.3533 - val_mse: 560.3533
Epoch 3/3
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 500.2576 - mse: 500.2576 - val_loss: 4034.7544 - val_mse: 4034.7544


KFOLD METHOD

In [97]:
# Import additionnal libraries

from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [96]:
X_kfold, X_val, y_kfold, y_val = train_test_split(X_train, y_train, test_size = 0.2, shuffle = True, random_state = 3)

In [99]:
# define model

def create_model(X, y, X_val, y_val, count) :
    
    callbacks = [
        EarlyStopping(monitor = "val_accuracy", patience = 20),
        ModelCheckpoint(
            filepath = "my_best_model" + str(count) + ".keras",
            monitor = "val_accuracy",
            mode = "max",
            save_best_only = True,
            verbose = 1
        )
    ]
        
    # Input layer
    x0 = [Input(shape = (1, )) for _ in range (16)]

    # Hiddens layer
    x1 = [Dense(3, input_dim = 1, activation = "relu")(x) for x in x0]

    x2 = [Dense(3, activation = "relu")(x) for x in x1]

    x3 = [Dense(3, activation = "relu")(x) for x in x2]

    x3_temp = x3.copy()

    x4 = [Concatenate()([x3[i], x3[i + 1]]) for i in range (0, len(x3), 2)]

    x5 = [Dense(6, activation = "relu")(x) for x in x4]

    x6 = [Dense(5, activation = "relu")(x) for x in x5]

    x7 = [Concatenate()([x6[i], x6[i + 1]] + x3_temp[2*i : int(4*(i/2 + 1))]) for i in range (0, len(x6), 2)]

    x8 = [Dense(8, activation = "relu")(x) for x in x7]

    x9 = [Dense(8, activation = "relu")(x) for x in x8]

    x10 = [Concatenate()([x9[i], x9[i + 1]]) for i in range (0, len(x9), 2)]

    x11 = [Dense(16, activation = "relu")(x) for x in x10]

    x12 = Concatenate()(x11)

    x13 = Dense(256, activation = "relu")(x12)

    # # Output layer
    x14 = Dense(1)(x13)

    # Create model
    model = Model(inputs = x0, outputs = x14)
    
    model.compile(optimizer = 'rmsprop', loss = ["mse"], metrics = ["mse"])
        
    model.fit(X, y, validation_data = (X_val, y_val), epochs = 3, batch_size = 32, verbose = 0, callbacks=callbacks)

    return model

In [101]:
# split data into 10 folds
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

# list that will store the results
cv_score = []

i = 0
for train_index, test_index in kfold.split(X_train, y_train) :
    # create model
    model = create_model([X_kfold[train_index][:, i] for i in range (16)], y_kfold[train_index], [X_kfold[test_index][:, i] for i in range (16)], y_kfold[test_index], i)
    
    # evaluate the model
    score = model.evaluate([X_kfold[test_index][:, i] for i in range (16)], y_kfold[test_index], verbose = 0)
    print(model.metrics_names[1], score[1]*100)
    cv_score.append(score[1]*100)
    i += 1
    
print("average accuracy and standard deviation:", np.mean(cv_score), np.std(cv_score))

IndexError: index 6400 is out of bounds for axis 0 with size 6400